In [ ]:
# Define a weighted loss function based on the variable choice

# Loss weights
weighted_loss_type = 'exponential'  # Options: 'exponential' or 'square'
weight_min = 300                  # Minimal freq_bin index to weight. Avoids weighting the near zero "peak".

def weighted_mse(y_true, y_pred):
    # Calculate weights based on the weighted_loss_type
    if weighted_loss_type == 'exponential':
        weights = tf.exp(y_true)
    elif weighted_loss_type == 'square':
        weights = tf.square(y_true)
    else:
        raise ValueError("Invalid weighted_loss_type. Choose 'exponential' or 'square'.")

    # Dynamically compute the shape to handle variable batch sizes
    batch_size = tf.shape(weights)[0]
    time_steps = tf.shape(weights)[1]

    # Create a mask for indices below the weight_min
    mask = tf.range(time_steps) < weight_min  # Shape: (time_steps,)
    mask = tf.expand_dims(mask, axis=0)       # Shape: (1, time_steps)
    mask = tf.expand_dims(mask, axis=-1)      # Shape: (1, time_steps, 1)
    mask = tf.tile(mask, [batch_size, 1, 1])  # Shape: (batch_size, time_steps, 1)

    # Replace weights below the cutoff with the minimum weight value
    min_weight = tf.reduce_min(weights[:, weight_min:, :], axis=1, keepdims=True)
    weights = tf.where(mask, tf.broadcast_to(min_weight, tf.shape(weights)), weights)

    # Compute the weighted mean squared error
    loss = tf.reduce_mean(weights * tf.square(y_true - y_pred))
    return loss


In [ ]:
import numpy as np
from pathlib import Path
import statsmodels.api as sm

# Get noise floor files

f = rfftfreq(32768, 1/44100)[0:-1]

"""Take FFT of all raw waveforms in the dataframe and convert to dB SPL"""
# Define constants that allow us to correct for experimental setup
amp_factor = 0.01  # Correct for amplifier gain
# amp_factor = 0.1  # Correct for amplifier gain
mic_factor = 10**(-6)  # Corresponds to the mic level of 1 microvolt
# mic_factor = 1  # Corresponds to the mic level of 1 microvolt
rms_factor = np.sqrt(2)  # Converts to RMS value

# Define window length
n_win = 32768

# First navigate to our directory
directory_path = Path(r"C:\Users\Owner\OneDrive\Desktop\Tube Noise Floors")
# directory_path = Path(r"C:\Users\Owner\OneDrive\Desktop\SOAE Data\Tube Files")

# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    if fp.is_file() == False:  
        continue
    elif "supp" in fp.name:
        noise_floor = np.loadtxt(fp)[:, 1]

    else:
        wf = np.loadtxt(fp)
        # Divide the waveform into windows of length n_win, take magnitude of FFT of each
        mags_list = [
            np.abs(rfft(wf[i * n_win:(i + 1) * n_win])*(2/n_win))
            for i in range(len(wf) // n_win)
        ]

        # Average over all windows
        avg_mags = np.mean(mags_list, axis=0)
        noise_floor = 20 * np.log10(avg_mags * amp_factor * rms_factor / mic_factor)
    
    
    lowess = sm.nonparametric.lowess
    # === deal w/ loess
    sigma= 0.08  # local-ness factor {0.1-0.2}
    fit= lowess(noise_floor,f/1000,frac=sigma, return_sorted=False)
    
    plt.plot(f, noise_floor)
    plt.plot(f, fit,'g--',label='loess fit')
    plt.title(fp.name)
    plt.show()
    


